In [4]:
!pip install langchain --quiet
!pip install openai --quiet

In [ ]:
import boto3

dynamodb = boto3.resource("dynamodb")

# 创建Dynamodb表
table = dynamodb.create_table(
    TableName="SessionTable",
    KeySchema=[{"AttributeName": "SessionId", "KeyType": "HASH"}],
    AttributeDefinitions=[{"AttributeName": "SessionId", "AttributeType": "S"}],
    BillingMode="PAY_PER_REQUEST",
)

# 等待回调结果
table.meta.client.get_waiter("table_exists").wait(TableName="SessionTable")

In [7]:
import os
os.environ['OPENAI_API_KEY']=open("openai-key").read()
os.environ["SERPAPI_API_KEY"]=open("serpapi-key").read()

In [21]:
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory

message_history = DynamoDBChatMessageHistory(table_name="SessionTable", session_id="2")
memory = ConversationBufferMemory(
    memory_key="chat_history", chat_memory=message_history, return_messages=True
)

In [22]:
from langchain import OpenAI, SerpAPIWrapper

search = SerpAPIWrapper()

tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

In [23]:
llm = ChatOpenAI(temperature=0)

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True
)

In [24]:
agent_chain.run(input="Hello, please introduce yourself shortly.")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "Hello! I am Assistant, a large language model trained by OpenAI. I am here to assist you with any questions or tasks you may have. How can I help you today?"
}

> Finished chain.


'Hello! I am Assistant, a large language model trained by OpenAI. I am here to assist you with any questions or tasks you may have. How can I help you today?'

In [25]:
agent_chain.run(input="who is the CEO of Amazon Company?")



> Entering new AgentExecutor chain...
{
    "action": "Intermediate Answer",
    "action_input": "CEO of Amazon Company"
}
Observation: Andy Jassy
Thought:{
    "action": "Final Answer",
    "action_input": "Andy Jassy is the CEO of Amazon Company."
}

> Finished chain.


'Andy Jassy is the CEO of Amazon Company.'

In [ ]:
agent_chain.run(input="OK. Tell me more about him please")



> Entering new AgentExecutor chain...
Could not parse LLM output: Andy Jassy is an American businessman who became the CEO of Amazon on July 5, 2021, succeeding Jeff Bezos. Prior to becoming CEO, Jassy served as the CEO of Amazon Web Services (AWS), the cloud computing division of Amazon. He joined Amazon in 1997 and played a key role in the development and growth of AWS, which has become a major player in the cloud computing industry.

Under Jassy's leadership, AWS experienced significant expansion and became a major revenue generator for Amazon. Jassy is known for his focus on customer-centric innovation and has been instrumental in driving the success of AWS by introducing new services and features to meet the evolving needs of businesses and developers.

Jassy holds a Bachelor of Arts in Government from Harvard University and a Master of Business Administration from Harvard Business School. He is widely recognized for his leadership and expertise in the technology industry.
Obser